In [1]:
import cv2
import numpy as np
import tensorflow as tf
import mediapipe as mp

In [18]:
# Initialize MediaPipe Holistic
mp_holistic = mp.solutions.holistic
holistic = mp_holistic.Holistic(static_image_mode=True, model_complexity=1, smooth_landmarks=True)

# Initialize MediaPipe Drawing
mp_drawing = mp.solutions.drawing_utils
drawing_spec = mp_drawing.DrawingSpec(color=(128,0,128),thickness=1, circle_radius=1)
connect_drawing_spec = mp_drawing.DrawingSpec(color=(200,200,200),thickness=1, circle_radius=1)

def mediapipe_detection(image, model):
    results = holistic.process(image)
    image_copy=image.copy()
    if results.pose_landmarks:
        mp_drawing.draw_landmarks(
            image_copy, results.pose_landmarks, mp_holistic.POSE_CONNECTIONS,
            landmark_drawing_spec=drawing_spec, connection_drawing_spec=drawing_spec)
    if results.face_landmarks:
        mp_drawing.draw_landmarks(
            image_copy, results.face_landmarks, mp_holistic.FACEMESH_TESSELATION,
            landmark_drawing_spec=drawing_spec, connection_drawing_spec=connect_drawing_spec)
    if results.left_hand_landmarks:
        mp_drawing.draw_landmarks(
            image_copy, results.left_hand_landmarks, mp_holistic.HAND_CONNECTIONS,
            landmark_drawing_spec=drawing_spec, connection_drawing_spec=drawing_spec)
    if results.right_hand_landmarks:
        mp_drawing.draw_landmarks(
            image_copy, results.right_hand_landmarks, mp_holistic.HAND_CONNECTIONS,
            landmark_drawing_spec=drawing_spec, connection_drawing_spec=drawing_spec)
    return image_copy, results

In [11]:
from tensorflow.keras.models import load_model
from tensorflow.keras.optimizers import Adam

# Step 1: Load the model without compiling it
model = load_model(r'C:\Users\kabou\Desktop\Nour\Courses\HCIB - AI\Project\9_Sign_Model.h5', compile=False)

# Step 2: Recompile the model with the Adam optimizer
model.compile(optimizer=Adam(), loss='categorical_crossentropy', metrics=['accuracy'])

In [13]:

# Define a function to preprocess input
def preprocess_input(frames):
    # Assuming frames is a list of image frames
    # Here you would extract landmarks or any other features needed for the model
    images, landmarks = extract_landmarks(frames)

    # Assuming the output of extract_landmarks is (num_frames, 21, 6)
    num_frames = len(landmarks)
    max_frames = 25  # The length used during training

    # If fewer frames, pad with zeros; if more, truncate
    if num_frames < max_frames:
        padding_length = max_frames - num_frames
        landmarks = np.pad(landmarks, ((0, padding_length), (0, 0), (0, 0)), 'constant', constant_values=0)
    elif num_frames > max_frames:
        landmarks = landmarks[:max_frames]

    landmarks = landmarks.reshape(1, max_frames, -1)
    return images,landmarks

# Function to extract landmarks from frames (example placeholder function)
def extract_keypoints(results):
    lh = np.array([[res.x, res.y, res.z] for res in results.left_hand_landmarks.landmark]).flatten() if results.left_hand_landmarks else np.zeros(21*3)
    rh = np.array([[res.x, res.y, res.z] for res in results.right_hand_landmarks.landmark]).flatten() if results.right_hand_landmarks else np.zeros(21*3)
    return np.concatenate([lh, rh])

In [43]:
import time
sequence = []
cap = cv2.VideoCapture(0)

# Main loop
while True:
    # Read feed
        ret, frame = cap.read()

        # Make detections
        image, results = mediapipe_detection(frame, holistic)
        
        # 2. Prediction logic
        keypoints = extract_keypoints(results)
        sequence.append(keypoints)
        sequence = sequence[-25:]
        
        if len(sequence) == 25:
            res = model.predict(np.expand_dims(sequence, axis=0),verbose=0)[0]
            predicted_class = np.argmax(res)
            predicted_class_value = res[predicted_class]
            if predicted_class_value>0.7:
                print(f"Predicted class: {predicted_class}")
                print(f"Predicted Class Value: {predicted_class_value}")

    # Display the frame
        cv2.imshow('Frame', image)
    
        # Break the loop if 'q' is pressed
        if cv2.waitKey(1) & 0xFF == ord('q'):
            break

# Release the capture and close windows
cap.release()
cv2.destroyAllWindows()